In [ ]:
import os

FOLDER = "swimdata/"

files = os.listdir(FOLDER)
files.remove(".DS_Store")

files

In [ ]:
first = files[0]
print(first)

In [ ]:
name, age, _, _ = first.removesuffix(".txt").split("-")
name, age

In [ ]:
import DBcm

db_details = "CoachDB.sqlite3"

SQL_INSERT = """
    insert into swimmers
    (name, age)
    values
    (?,?)
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_INSERT, (name, age))

In [ ]:
SQL_SELECT = """
    select * from swimmers
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_SELECT)
    result = db.fetchall()
    print(result)

In [ ]:
SQL_DELETE = """
    delete from swimmers
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_DELETE);


In [ ]:
with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        name, age , *_ = fn.removesuffix(".txt").split("-")
        db.execute(SQL_INSERT, (name, age))


In [ ]:
SQL_SELECT = """
     select * from swimmers
     where name = ? and age = ?
"""

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        name, age, _, _ = fn.removesuffix(".txt").split("-")
        db.execute(SQL_SELECT, (name, age))
        if db.fetchall():
            continue
        db.execute(SQL_INSERT, (name, age))

In [ ]:
SQL_SELECT_EVENTS = """
     select * from events
     where distance = ? and stroke = ?
"""

SQL_INSERT_EVENTS = """
    insert into events
    (distance, stroke)
    values
    (?,?)
"""

SQL_DELETE_EVENTS = """
    delete from events
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_DELETE_EVENTS);

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        _, _, distance, stroke = fn.removesuffix(".txt").split("-")
        db.execute(SQL_SELECT_EVENTS, (distance, stroke))
        if db.fetchall():
            continue
        db.execute(SQL_INSERT_EVENTS, (distance, stroke))

In [ ]:
SQL_SELECT_EVENTS = """
    select * from events
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_SELECT_EVENTS)
    result = db.fetchall()
    print(result)

In [ ]:
SQL_GET_SWIMMER = """
    select id from swimmers
    where name = ?  and age = ? 
"""

SQL_GET_EVENT = """
    select id from events
    where distance = ? and stroke = ?
"""

SQL_INSERT_TIMES = """
    insert into times
    (swimmer_id, event_id, time)
    values
    (?, ?, ?)
"""

SQL_DELETE_TIMES = """
    delete from times
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_DELETE_TIMES);

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        name, age, distance, stroke = fn.removesuffix(".txt").split("-")
        db.execute(SQL_GET_SWIMMER, (name, age))
        swimmer_id, *_ = db.fetchall()[0]
        db.execute(SQL_GET_EVENT, (distance, stroke))
        event_id, *_ = db.fetchall()[0]
        print(swimmer_id, event_id)
        with open(FOLDER+fn) as sf:
            times = sf.read().strip().split(",")
            for t in times:
                db.execute(SQL_INSERT_TIMES, (swimmer_id, event_id, t))

SQL_GET_TIMES = """
    select swimmer_id, event_id, time, ts from times
"""
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_GET_TIMES)
    result = db.fetchall()
    print(result)
